In [4]:
# ==============================
# CS-340 Project Two Dashboard
# Codio-friendly: JupyterDash INLINE + MongoDB via AnimalShelter CRUD
# ==============================

# If needed (run once):
# !pip -q install jupyter-dash

import pandas as pd
import plotly.express as px

from jupyter_dash import JupyterDash
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output

from CRUD_Python_Module import AnimalShelter


# ------------------------------
# 1) CONNECT TO MONGODB
# ------------------------------
shelter = AnimalShelter("aacuser", "Password123!")


# ------------------------------
# 2) FILTER DEFINITIONS
# ------------------------------
WATER_BREEDS = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
MOUNTAIN_BREEDS = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
DISASTER_BREEDS = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]

AGE_RANGE = {"$gte": 26, "$lte": 156}


def mongo_query(filter_value: str) -> dict:
    if filter_value == "Reset":
        return {}

    if filter_value == "Water Rescue":
        return {
            "animal_type": "Dog",
            "breed": {"$in": WATER_BREEDS},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": AGE_RANGE,
        }

    if filter_value == "Mountain/Wilderness":
        return {
            "animal_type": "Dog",
            "breed": {"$in": MOUNTAIN_BREEDS},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": AGE_RANGE,
        }

    if filter_value == "Disaster/Tracking":
        return {
            "animal_type": "Dog",
            "breed": {"$in": DISASTER_BREEDS},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": AGE_RANGE,
        }

    return {}


# ------------------------------
# 3) APP SETUP
# ------------------------------
JupyterDash.infer_jupyter_proxy_config()
app = JupyterDash(__name__)


def load_df(query: dict) -> pd.DataFrame:
    data = shelter.read(query)
    dff = pd.DataFrame(list(data))

    if dff.empty:
        return dff

    if "_id" in dff.columns:
        dff["_id"] = dff["_id"].astype(str)

    return dff


def empty_scatter(title: str):
    fig = px.scatter(pd.DataFrame({"x": [], "y": []}), x="x", y="y", title=title)
    fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))
    return fig


df_start = load_df({})


# ------------------------------
# 4) LAYOUT
# ------------------------------
app.layout = html.Div(
    style={"padding": "12px", "fontFamily": "Arial"},
    children=[
        html.H2("Grazioso Salvare Dashboard"),
        html.Div("Developer: Liseth Marquez Torres", style={"marginBottom": "10px"}),

        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Reset (All)", "value": "Reset"},
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountain/Wilderness Rescue", "value": "Mountain/Wilderness"},
                {"label": "Disaster Rescue / Individual Tracking", "value": "Disaster/Tracking"},
            ],
            value="Reset",
            inline=True,
            style={"marginBottom": "12px"},
        ),

        html.Div(
            style={"display": "grid", "gridTemplateColumns": "1.2fr 1fr", "gap": "12px"},
            children=[
                dash_table.DataTable(
                    id="datatable",
                    columns=[{"name": c, "id": c} for c in df_start.columns],
                    data=df_start.to_dict("records"),
                    page_size=10,
                    sort_action="native",
                    filter_action="native",
                    style_table={"overflowX": "auto", "maxHeight": "650px", "overflowY": "auto"},
                    style_cell={"fontSize": "12px", "padding": "6px", "textAlign": "left"},
                ),

                html.Div(
                    children=[
                        dcc.Graph(id="geo-map"),
                        dcc.Graph(id="breed-pie"),
                    ]
                ),
            ],
        ),
    ],
)


# ------------------------------
# 5) CALLBACK: FILTER -> TABLE + CHARTS
# ------------------------------
@app.callback(
    Output("datatable", "columns"),
    Output("datatable", "data"),
    Output("geo-map", "figure"),
    Output("breed-pie", "figure"),
    Input("filter-type", "value"),
)
def update_dashboard(filter_value):
    query = mongo_query(filter_value)
    dff = load_df(query)

    # TABLE
    if dff.empty:
        cols = []
        data = []
    else:
        cols = [{"name": c, "id": c} for c in dff.columns]
        data = dff.to_dict("records")

    # MAP
    if not dff.empty and "location_lat" in dff.columns and "location_long" in dff.columns:
        dff["location_lat"] = pd.to_numeric(dff["location_lat"], errors="coerce")
        dff["location_long"] = pd.to_numeric(dff["location_long"], errors="coerce")
        map_df = dff.dropna(subset=["location_lat", "location_long"])

        if map_df.empty:
            geo_fig = empty_scatter("No map data for this filter")
        else:
            geo_fig = px.scatter_mapbox(
                map_df,
                lat="location_lat",
                lon="location_long",
                hover_name="name" if "name" in map_df.columns else None,
                zoom=9,
                title="Animal Locations (Filtered)",
            )
            geo_fig.update_layout(mapbox_style="open-street-map", margin=dict(l=0, r=0, t=40, b=0))
    else:
        geo_fig = empty_scatter("No location_lat/location_long fields found (or no records)")

    # PIE
    if not dff.empty and "breed" in dff.columns:
        top = dff["breed"].value_counts().head(10).reset_index()
        top.columns = ["breed", "count"]
        pie_fig = px.pie(top, names="breed", values="count", title="Top 10 Breeds (Filtered)")
        pie_fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))
    else:
        pie_fig = px.pie(
            pd.DataFrame({"breed": ["No Data"], "count": [1]}),
            names="breed",
            values="count",
            title="Top 10 Breeds (Filtered)",
        )
        pie_fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

    return cols, data, geo_fig, pie_fig


# ------------------------------
# 6) RUN (INLINE)
# ------------------------------
app.run_server(mode="inline", host="0.0.0.0", port=8051, debug=False)



 * Running on all addresses.
 * Running on http://10.56.165.94:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2025 20:51:22] "GET /_alive_3b03a018-5f9a-473d-85a8-ef7c96d044a7 HTTP/1.1" 200 -


127.0.0.1 - - [12/Dec/2025 20:51:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2025 20:51:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2025 20:51:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2025 20:51:26] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2025 20:51:26] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2025 20:51:26] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2025 20:51:26] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2025 20:51:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2025 20:53:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2025 20:53:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2025 20:53:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - -